In [1]:

import numpy as np 
import pandas as pd
import tracers as tr
import plotly.graph_objects as go
from GeoPrivacy.mechanism import random_laplace_noise
from pyproj import Transformer

## Raw data

One taxi in the romataxi crawdad data set

In [2]:
df = pd.read_csv('taxidata/taxi_156.csv')
# covert timestamp to epoch
df['Timestamp'] = pd.to_datetime(df['Timestamp']).astype(int) / 10**9
# sort by time
df = df.sort_values(by='Timestamp')
# to array on (lon,lat)
trace = df[['Longitude','Latitude','Timestamp']].to_numpy()
# take only 1000 points
trace = trace[:1000]
# trace holds tuples instead of arrays
trace = [tuple(x) for x in trace]
#time float to int
trace = [(x[0],x[1],int(x[2])) for x in trace]


In [ ]:
fig = go.Figure()
fig.add_trace(go.Scattermapbox(
    lat=[b for a,b,c in trace],
    lon=[a for a,b,c in trace],
    mode='lines+markers',
    marker=go.scattermapbox.Marker(
        size=9
    ),
    text=df['Timestamp']
))
center_of_rome = [41.9028, 12.4964]
fig.update_layout(mapbox_style="open-street-map",mapbox_center_lon=center_of_rome[1], mapbox_center_lat=center_of_rome[0],mapbox_zoom=10)
fig.show()

## Geo-indistinguishability using planar Laplace mechanism
https://github.com/quao627/GeoPrivacy/blob/main/GeoPrivacy/mechanism.py

In [4]:
eps = 0.05
noisy_trace = []

# Italy in UTM 33N 
utm_to_wgs84 = Transformer.from_crs(3065, 4326, always_xy=True)
wgs84_to_utm = Transformer.from_crs(4326, 3065, always_xy=True)


for i in trace:
    noise = random_laplace_noise(eps)  
    x,y = wgs84_to_utm.transform(i[1],i[0]) 
    noisy_x = x + noise[0]
    noisy_y = y + noise [1]

    noisy_lat, noisy_lon = utm_to_wgs84.transform(noisy_x,noisy_y) 
    noisy_point = (noisy_lon,noisy_lat,i[2])
    noisy_trace.append(noisy_point) 



In [ ]:
fig.add_trace(go.Scattermapbox(
    lon=[a for a, b, c in noisy_trace],
    lat=[b for a, b, c in noisy_trace],
    mode='lines+markers',
    line_color="green",
    marker=go.scattermapbox.Marker(size=9),
))